# File Unzip Utility

Extracts zip files in-place (to the same directory as the zip file).

**Instructions:** Update the `zip_filename` below and run all cells.

The utility searches for the zip file in this order:
1. Active cycle: `workflows/Active_{cycle}/files/data/`
2. _Template: `workflows/_Template/files/data/`
3. _Tools: `workflows/_Tools/files/data/`

## Configuration

Update the filename (just the filename, not the full path):

In [1]:
# UPDATE THIS TO YOUR ZIP FILENAME
zip_filename = "Import Files 202509 xUSFL.zip"

# Delete zip file after extraction? (True/False)
delete_after_extract = False

## Extract Zip File

In [2]:
import zipfile
import sys
from pathlib import Path

# Add workspace to path
sys.path.append('../../')
from helpers.cycle import get_active_cycle

# Determine the correct files/data directory
workspace_root = Path.cwd()
while workspace_root.name != 'workspace' and workspace_root.parent != workspace_root:
    workspace_root = workspace_root.parent

if workspace_root.name != 'workspace':
    raise FileNotFoundError("Could not find workspace directory")

# Build list of directories to search (in priority order)
search_dirs = []

# 1. Try active cycle first
try:
    active_cycle = get_active_cycle()
    if active_cycle:
        cycle_name = active_cycle['cycle_name']
        search_dirs.append(workspace_root / "workflows" / f"Active_{cycle_name}" / "files" / "data")
except Exception:
    pass

# 2. Then _Template
search_dirs.append(workspace_root / "workflows" / "_Template" / "files" / "data")

# 3. Finally _Tools
search_dirs.append(workspace_root / "workflows" / "_Tools" / "files" / "data")

# Search for zip file in each directory
zip_path = None
for data_dir in search_dirs:
    candidate = data_dir / zip_filename
    if candidate.exists() and candidate.is_file():
        zip_path = candidate
        print(f"Found zip file at: {zip_path}")
        break

# If not found, raise error with search locations
if zip_path is None:
    search_locations = "\n  ".join([str(d) for d in search_dirs])
    raise FileNotFoundError(
        f"Zip file '{zip_filename}' not found in any of these locations:\n  {search_locations}"
    )

# Extract to same directory as zip file
extract_dir = zip_path.parent

print(f"\nExtracting: {zip_path.name}")
print(f"To directory: {extract_dir}")

extracted_files = []

try:
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        file_list = zip_ref.namelist()
        print(f"Archive contains {len(file_list)} file(s)\n")
        
        # Extract all files
        for i, file_name in enumerate(file_list, 1):
            print(f"  [{i}/{len(file_list)}] {file_name}")
            zip_ref.extract(file_name, extract_dir)
            extracted_files.append(extract_dir / file_name)
        
        print(f"\n✓ Extraction complete: {len(extracted_files)} file(s) extracted")

except zipfile.BadZipFile as e:
    raise zipfile.BadZipFile(f"Invalid zip file: {zip_path}") from e

# Delete zip file if requested
if delete_after_extract:
    try:
        zip_path.unlink()
        print(f"✓ Deleted zip file: {zip_path.name}")
    except PermissionError as e:
        print(f"✗ Warning: Cannot delete zip file (may be in use): {zip_path}")

print(f"\nDone!")

Found zip file at: /home/jovyan/workspace/workflows/_Template/files/data/Import Files 202509 xUSFL.zip

Extracting: Import Files 202509 xUSFL.zip
To directory: /home/jovyan/workspace/workflows/_Template/files/data
Archive contains 17 file(s)

  [1/17] 02 Import Files and Shapes/
  [2/17] 02 Import Files and Shapes/Modeling_202509_Moodys_CBEQ_Account.csv
  [3/17] 02 Import Files and Shapes/Modeling_202509_Moodys_CBEQ_Location.csv
  [4/17] 02 Import Files and Shapes/Modeling_202509_Moodys_CBHU_Account.csv
  [5/17] 02 Import Files and Shapes/Modeling_202509_Moodys_CBHU_Location.csv
  [6/17] 02 Import Files and Shapes/Modeling_202509_Moodys_USEQ_Account.csv
  [7/17] 02 Import Files and Shapes/Modeling_202509_Moodys_USEQ_Location.csv
  [8/17] 02 Import Files and Shapes/Modeling_202509_Moodys_USFF_Account.csv
  [9/17] 02 Import Files and Shapes/Modeling_202509_Moodys_USFF_Location.csv
  [10/17] 02 Import Files and Shapes/Modeling_202509_Moodys_USHU_Full_Account.csv
  [11/17] 02 Import Files 